# Creating a STAC Item for the NOAA HRRR Forecast (dynamical.org Icechunk)

This notebook shows how to use `cloudify.stac` to generate a STAC item for the
[NOAA HRRR 48-Hour Forecast](https://dynamical.org/catalog/noaa-hrrr-forecast-48-hour/)
dataset published by [dynamical.org](https://dynamical.org) as a public Icechunk store on AWS S3.

HRRR differs from GFS in one important way: it uses a **Lambert Conformal Conic
projection** with `x`/`y` coordinates in metres, not `latitude`/`longitude`.
This means:
- `rioxarray` is required to reproject the native bounds to WGS84 for the STAC bbox
- `x_dimension="x"`, `y_dimension="y"` must be passed explicitly
- `reference_system` should reflect the native CRS (read from `ds.rio.crs`)

## Install dependencies
```
pip install icechunk xarray zarr pystac xstac rioxarray pyproj cloudify
```

In [ ]:
import json

import icechunk
import pystac
import rioxarray  # registers .rio accessor; required for CRS-aware bbox
import xarray as xr

from cloudify.stac import build_stac_item_from_icechunk

## 1. Open the Icechunk store

In [ ]:
BUCKET = "dynamical-noaa-hrrr"
PREFIX = "noaa-hrrr-forecast-48-hour/v0.1.0.icechunk/"
REGION = "us-west-2"
ICECHUNK_HREF = f"s3://{BUCKET}/{PREFIX}"

storage = icechunk.s3_storage(
    bucket=BUCKET,
    prefix=PREFIX,
    region=REGION,
    anonymous=True,
)
repo = icechunk.Repository.open(storage)
session = repo.readonly_session("main")

snapshot_id = session.snapshot_id
print(f"snapshot_id: {snapshot_id}")

In [ ]:
ds = xr.open_zarr(session.store, chunks=None)
ds

## 2. Inspect the CRS

HRRR uses a Lambert Conformal Conic projection. `ds.rio.crs` gives us the
native CRS; `extract_spatial_extent_rio` will automatically reproject the
dataset bounds to WGS84 (EPSG:4326) for the STAC bbox.

In [ ]:
crs = ds.rio.crs
print(f"Native CRS: {crs}")

# HRRR's Lambert Conformal Conic projection has no standard EPSG code,
# so to_epsg() returns None — fall back to the full WKT string.
reference_system = crs.to_epsg() or crs.to_wkt()
print(f"reference_system type: {'EPSG int' if isinstance(reference_system, int) else 'WKT string'}")

## 3. Define storage schemes and providers

In [ ]:
storage_schemes = {
    "aws-s3-dynamical-noaa-hrrr": {
        "type": "aws-s3",
        "platform": "https://{bucket}.s3.{region}.amazonaws.com",
        "bucket": BUCKET,
        "region": REGION,
        "anonymous": True,
    }
}

providers = [
    pystac.Provider(
        name="dynamical.org",
        description="Analysis-ready, cloud-optimized weather forecast data",
        roles=["producer", "processor", "host"],
        url="https://dynamical.org",
    ),
    pystac.Provider(
        name="NOAA NCEP",
        description="National Oceanic and Atmospheric Administration — High-Resolution Rapid Refresh",
        roles=["producer", "licensor"],
        url="https://www.ncep.noaa.gov",
    ),
]

## 4. Build the STAC item

Key arguments for HRRR that differ from a plain lat/lon dataset:

| Argument | Value | Reason |
|---|---|---|
| `x_dimension` | `"x"` | Projected easting coordinate in metres |
| `y_dimension` | `"y"` | Projected northing coordinate in metres |
| `temporal_dimension` | `"init_time"` | Forecast initialisation time |
| `reference_system` | native CRS EPSG | Preserves projection info in the datacube extension |
| `virtual` | `False` | Regular Icechunk store (not virtual-chunk references) |

`extract_spatial_extent_rio` (called internally) uses
`rioxarray.transform_bounds("EPSG:4326")` to reproject the native Lambert
Conformal bounds to WGS84, which is required for a valid STAC bbox.

In [ ]:
item_id = f"noaa-hrrr-forecast-48-hour-{snapshot_id.lower()}"

item = build_stac_item_from_icechunk(
    ds,
    item_id=item_id,
    icechunk_href=ICECHUNK_HREF,
    snapshot_id=snapshot_id,
    storage_schemes=storage_schemes,
    title="NOAA HRRR 48-Hour Forecast (dynamical.org)",
    description=ds.attrs.get("description",
        "NOAA High-Resolution Rapid Refresh (HRRR) 48-hour forecast data, "
        "analysis-ready and cloud-optimized by dynamical.org. "
        "Continental US coverage at 3 km resolution in Lambert Conformal "
        "Conic projection, initialized every 6 hours from 2018-07-13 to present."
    ),
    providers=providers,
    virtual=False,                    # regular icechunk, not virtual
    temporal_dimension="init_time",   # forecast initialisation time
    x_dimension="x",                  # projected easting (metres)
    y_dimension="y",                  # projected northing (metres)
    reference_system=reference_system,
)

print(json.dumps(item, indent=2))

## 5. Inspect key fields

In [ ]:
print("bbox (WGS84):   ", item["bbox"])
print("start_datetime: ", item["properties"]["start_datetime"])
print("end_datetime:   ", item["properties"]["end_datetime"])
print("variables:      ", list(item["properties"]["cube:variables"].keys()))
print()
print("assets:")
for key, asset in item["assets"].items():
    print(f"  {key}: {asset['href']}")
    print(f"    snapshot_id: {asset.get('icechunk:snapshot_id')}")
    print(f"    roles:       {asset['roles']}")
print()
print("storage:schemes at top level:", "storage:schemes" in item)
print("storage:schemes in properties (should be False):",
      "storage:schemes" in item.get("properties", {}))

## 6. Save and round-trip via xpystac

In [ ]:
out_path = f"{item_id}.json"
with open(out_path, "w") as f:
    json.dump(item, f, indent=2)
print(f"Written to {out_path}")

In [ ]:
# Reload and open via xpystac  (requires: pip install xpystac)
loaded_item = pystac.Item.from_file(out_path)

asset_key = next(k for k in loaded_item.assets if "@" in k)
asset = loaded_item.assets[asset_key]
print(f"Opening asset: {asset_key}")

ds_from_stac = xr.open_dataset(asset)
ds_from_stac